In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage import io

import drawMatches as dm

In [9]:
# Constants
METHOD_ORB = 'orb'
METHOD_SIFT = 'sift'
METHOD_SURF = 'surf'

In [119]:
# Hyper params
SCORE_FACTOR_ORB = 1500
SCORE_FACTOR_SIFT = 1000
SCORE_FACTOR_SURF = 0.1

SCORE_FACTOR = {}
SCORE_FACTOR[METHOD_ORB] = SCORE_FACTOR_ORB
SCORE_FACTOR[METHOD_SIFT] = SCORE_FACTOR_SIFT
SCORE_FACTOR[METHOD_SURF] = SCORE_FACTOR_SURF


In [23]:
orb = cv2.ORB()
sift = cv2.SIFT()
surf = cv2.SURF(400)

In [112]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [13]:
def addSetToList(files, name):
    files.extend(map(lambda x: '/data/img/' + name + '/' + x, [f for f in os.listdir(os.getcwd() + '/data/img/' + name) if os.path.isfile(os.path.join(os.getcwd() + '/data/img/' + name, f))]))

In [122]:
def loadSets():
    cardList = []
    
    addSetToList(cardList, 'KLD')
    return cardList

In [113]:
def resetScores(cardData):
    for name in cardData:
        cardData[name][2] = 0

In [151]:
def getCardScores(descriptor, cardData, allDescriptors, cardIndex, featureMethod):
    matches =  findGoodMatches(descriptor, allDescriptors, featureMethod)
    
    for match in matches:
        cardData[cardIndex[match.trainIdx]][2] += (1 / (match.distance ** 2))

In [16]:
def smoothImage(image):
    """
    Smoothes an image with a gaussian filter.
    The filter size used is 5x5.
    """
    return cv2.GaussianBlur(image, (5, 5), 0)

In [17]:
def prepareImage(image):
    """
    Preprocesses an image before the features are calculated.
    
    The images gets resized to 128x128 and becomes a grayscale image.
    """
    image =  cv2.resize(image, (128, 128), interpolation = cv2.INTER_CUBIC)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image

In [18]:
def getKeypointsAndDescriptor(image, featureMethod):
    """
    Calculates the keypoints and their descriptors for the given image using
    the method that was passed.
    """
    if(featureMethod == METHOD_ORB):
        return orb.detectAndCompute(image, None)
    elif(featureMethod == METHOD_SIFT):
        return sift.detectAndCompute(image, None)
    elif(featureMethod == METHOD_SURF):
        return surf.detectAndCompute(image, None)

In [128]:
def findMatches(descriptor1, descriptor2, featureMethod):
    """
    Finds matches between the given lists of descriptors.
    Uses the method passed.
    """
    if(featureMethod == METHOD_ORB):
        bf = cv2.BFMatcher(cv2.NORM_HAMMING)
        return bf.knnMatch(descriptor1, descriptor2, k=2)
    if(featureMethod == METHOD_SIFT):
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary

        flann = cv2.FlannBasedMatcher(index_params,search_params)

        return flann.knnMatch(descriptor1,descriptor2,k=2)
    elif(featureMethod == METHOD_SURF):
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary

        flann = cv2.FlannBasedMatcher(index_params,search_params)

        return flann.knnMatch(descriptor1,descriptor2,k=2)

In [130]:
def findGoodMatches(descriptor1, descriptor2, featureMethod):
    matches = findMatches(descriptor1, descriptor2, featureMethod)
    
    goodMatches = []

    for i in range(0, len(matches)):
        # We only want matches between two descriptors
        if(len(matches[i]) == 2):
            m = matches[i][0]
            n = matches[i][1]
            #D.Lowe's ratio test 
            if m.distance < 1 * n.distance:
                goodMatches.append(m)
    return goodMatches

In [153]:
def findBestMatch(cardImg, cardData, allDescriptors, cardIndex, featureMethod):
    cardImg = prepareImage(cardImg)
    
    keypoints, descriptor = getKeypointsAndDescriptor(cardImg, featureMethod)
    
    getCardScores(descriptor, cardData, allDescriptors, cardIndex, featureMethod)
    
    i = 0
    scores = np.zeros((len(cardData)))
    
    for card  in cardData:
        factor = SCORE_FACTOR[featureMethod]
        
        scores[i] = cardData[card][2] * factor
        i = i + 1
        
    softMaxScores = softmax(scores)
    softMaxValue = np.max(softMaxScores)
    
    print scores
    print 'Softmax', softMaxValue
    print jk
    sortedNames = sorted(cardData, key=lambda x: cardData[x][2])
    bestMatchCardName = sortedNames[len(sortedNames) - 1]
    
    cardData[bestMatchCardName][3] = softMaxValue
    
    return bestMatchCardName

In [108]:
def computeKeypoints(cardList, featureMethod):
    cardData = {}
    allDescriptors = None
    cardIndex = []
    
    print 'Started computing keypoints for ', featureMethod
    
    for fileName, i in zip(cardList, range(0, len(cardList))):
        img = cv2.imread(os.getcwd() + fileName)
        img = img[36:174, 18:206]
        img = prepareImage(img)
        img = smoothImage(img)
        
        keypoints, descriptor = getKeypointsAndDescriptor(img, featureMethod)
        
        if allDescriptors is None:
            allDescriptors = np.array(descriptor)
        else:
            allDescriptors = np.concatenate([allDescriptors, descriptor])
            
        for i in range(0, descriptor.shape[0]):
            cardIndex.append(fileName)
        # cardData[0] = keypoints
        # cardData[1] = descriptor
        # cardData[2] = similarityScore
        # cardDate[3] = softMaxScore
        cardData[fileName] = [keypoints, descriptor, 0, 0]
        
    print 'Finished computing keypoints for ', featureMethod
    
    return cardData, allDescriptors, cardIndex

In [145]:
def runTest(featureMethod, cardData, allDescriptors, cardIndex, testSet):
    correctMatches = []
    missClassified = []
    lowConfidence = []
    
    for testCard in testSet:
        resetScores(cardData)
        
        testCardImg = cv2.imread(os.getcwd() + testCard)
        testCardImg = testCardImg[0:70, 0:85]
        
        bestMatch = findBestMatch(testCardImg, cardData, allDescriptors, cardIndex, featureMethod)
        print bestMatch
        

In [146]:
dataOrb = computeKeypoints(cardList, METHOD_ORB)

Started computing keypoints for  orb
Finished computing keypoints for  orb


In [147]:
testCards = loadSets()


In [154]:
runTest(METHOD_ORB, dataOrb[0], dataOrb[1], dataOrb[2], testCards)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

NameError: global name 'jk' is not defined